In [ ]:
from pathlib import Path
import pandas as pd
import altair as alt
from scipy.stats.mstats import gmean
import numpy as np
import re

In [ ]:
# Combinations of width configuration mode and partial clock gating on/off.

power_modes = {'power':False, 'power_clkgated':True}

modes = {
    'Original'      :{'Fuse':False, 'Penalty':False, 'Gated':False},
    'Packing'       :{'Fuse':False, 'Penalty':False, 'Gated':True},
    'PackingPenalty':{'Fuse':False, 'Penalty':True, 'Gated':True},
    'Fusing'        :{'Fuse':True, 'Penalty':False, 'Gated':True},
    'FusingPenalty' :{'Fuse':True, 'Penalty':True, 'Gated':True}
}
modes_order = {'Original':0, 'Packing':1, 'PackingPenalty':2, 'Fusing':3, 'FusingPenalty':4}
modes_nopenalty = ['Original', 'Packing', 'Fusing']
modes_penalty = ['Original', 'PackingPenalty', 'FusingPenalty']

df_modes = pd.DataFrame.from_dict(modes, orient='index') 
df_modes = df_modes.rename_axis('Mode').reset_index()


# Reference config for each CPU.

cpus     = ['A76', 'HP']
ref_mode = 'Original'
ref_fu   = {'A76':2, 'HP':3}

In [ ]:
# Benchmarks to include, and their name in the plot.

benchmarks = {
    #'hmmer-small_1':'hmmer',
    #'fft_simsmall_1':'fft-splash',
    'integerNN_1':'intNN',
    'streamvbyte_1':'streamvbyte',
    'img_cartoon_tiny_image1_50per':'cartoon',
    'img_cartoon_tiny_image2_50per':'cartoon',
    'img_cartoon_tiny_image3_50per':'cartoon',
    'img_cartoon_tiny_image4_50per':'cartoon',
    'img_cartoon_tiny_image5_50per':'cartoon',
    'img_canny_tiny_image1_50per':'canny',
    'img_canny_tiny_image2_50per':'canny',
    'img_canny_tiny_image3_50per':'canny',
    'img_canny_tiny_image4_50per':'canny',
    'img_canny_tiny_image5_50per':'canny',
    'img_hist_tiny_image1_50per':'hist',
    'img_hist_tiny_image2_50per':'hist',
    'img_hist_tiny_image3_50per':'hist',
    'img_hist_tiny_image4_50per':'hist',
    'img_hist_tiny_image5_50per':'hist',
    'img_integral_tiny_image1_50per':'img_integral',
    'img_integral_tiny_image2_50per':'img_integral',
    'img_integral_tiny_image3_50per':'img_integral',
    'img_integral_tiny_image4_50per':'img_integral',
    'img_integral_tiny_image5_50per':'img_integral',
    'conv_tiny_image1_50per':'conv',
    'conv_tiny_image2_50per':'conv',
    'conv_tiny_image3_50per':'conv',
    'conv_tiny_image4_50per':'conv',
    'conv_tiny_image5_50per':'conv',
    'img_erode_tiny_image1_50per':'erode',
    'img_erode_tiny_image2_50per':'erode',
    'img_erode_tiny_image3_50per':'erode',
    'img_erode_tiny_image4_50per':'erode',
    'img_erode_tiny_image5_50per':'erode',
    'img_median_tiny_image1_50per':'median',
    'img_median_tiny_image2_50per':'median',
    'img_median_tiny_image3_50per':'median',
    'img_median_tiny_image4_50per':'median',
    'img_median_tiny_image5_50per':'median',
    'amax_cols_tiny_16bit_0':'amax',
    'amax_cols_tiny_16bit_1':'amax',
    'amax_cols_tiny_16bit_2':'amax',
    'amax_cols_tiny_16bit_3':'amax',
    'amax_cols_tiny_16bit_4':'amax',
    #'asum_cols_tiny_16bit_0':'asum',
    #'asum_cols_tiny_16bit_1':'asum',
    #'asum_cols_tiny_16bit_2':'asum',
    #'asum_cols_tiny_16bit_3':'asum',
    #'asum_cols_tiny_16bit_4':'asum',
    'gemv_tiny_16bit_0':'gemv',
    'gemv_tiny_16bit_1':'gemv',
    'gemv_tiny_16bit_2':'gemv',
    'gemv_tiny_16bit_3':'gemv',
    'gemv_tiny_16bit_4':'gemv',
    #'ger_tiny_16bit_0':'ger',
    #'ger_tiny_16bit_1':'ger',
    #'ger_tiny_16bit_2':'ger',
    #'ger_tiny_16bit_3':'ger',
    #'ger_tiny_16bit_4':'ger',
    'sqnrm2_cols_tiny_16bit_0':'sqnrm2',
    'sqnrm2_cols_tiny_16bit_1':'sqnrm2',
    'sqnrm2_cols_tiny_16bit_2':'sqnrm2',
    'sqnrm2_cols_tiny_16bit_3':'sqnrm2',
    'sqnrm2_cols_tiny_16bit_4':'sqnrm2',
    'gemm_tiny_16bit_0':'gemm',
    'gemm_tiny_16bit_1':'gemm',
    'gemm_tiny_16bit_2':'gemm',
    'gemm_tiny_16bit_3':'gemm',
    'gemm_tiny_16bit_4':'gemm',
    'fft_tiny_8bit_0':'fft',
    'fft_tiny_8bit_1':'fft',
    'fft_tiny_8bit_2':'fft',
    'fft_tiny_8bit_3':'fft',
    'fft_tiny_8bit_4':'fft',
}

bench_order = ['hmmer', 'fft-splash', 'intNN', 'streamvbyte', 'cartoon', 'sqnrm2', 'amax', 'gemv', 'gemm', 'fft', 'conv', 'median', 'img_integral', 'hist', 'canny', 'erode']

In [ ]:
# Parameter filtering.

component = 'fp/simd alu'

In [ ]:
# Plot labels.

labels = {'Dynamic':'Dynamic Energy Reduction [%]',
          'Leakage':'Leakage Energy Reduction [%]',
          'Total':'Total Energy Reduction [%]',
          'Time':'Normalized Execution Time'
          }

In [ ]:
# List all the run output folder paths.

folder = './stats'

runs = list(Path(folder).glob('**/roi.txt'))
runs = [r.parents[0] for r in runs]
#runs

In [ ]:
# Create table with the energy and time results for the specified runs.

df = []

for r in runs:
    # Params that can only be obtained from the path.
    cpu = r.parents[1].name.split('.')[0]
    pen = int(re.findall('\d?(?=pen)', r.parents[1].name)[0]) > 0
    
    # Params that can be obtained from config.csv.
    try:
        param = pd.read_csv(r/'config.csv', header=0)
    except:
        continue
    fu = param.iloc[0]['simdCount']
    fuse = param.iloc[0]['simdFuseCap'] > 0
    wBlock = int(param.iloc[0]['widthBlockSize'])
    bench = Path(param.iloc[0]['bootscript']).stem
    
    if cpu in cpus and bench in benchmarks.keys():
        # Time results.
        time = pd.read_csv(r/'time.csv', header=0)
        try:
            time = float(time.iloc[0]['sim seconds'])
        except TypeError:
            continue
        except IndexError:
            continue

        # Mcpat results. (FIXME: Can change here for more components!)
        for stem, gated in power_modes.items():
            mcpat = pd.read_csv(r/'{}.csv'.format(stem))
            mcpat = mcpat[mcpat['Component'] == component]

            dynamic = float(mcpat.loc[:, 'Runtime Dynamic [W]'].iloc[0] * time)
            leakage = float((mcpat.loc[:, ['Gate Leakage [W]', 'Subthreshold Leakage [W]']].sum(axis='columns')).iloc[0]) * time

            # Add entry to the table.
            df.append({'Benchmark':bench, 'CPU':cpu, 'SIMD FU':fu, 'Fuse':fuse, 'Penalty':pen, 'Width Block':wBlock,
                       'Gated':gated, 'Time':time, 'Leakage':leakage, 'Dynamic':dynamic, 'Total':dynamic+leakage})
    
df = pd.DataFrame(df)
df = df.merge(df_modes, how='right', on=['Fuse', 'Penalty', 'Gated'])
df = df.drop(columns=['Fuse', 'Penalty', 'Gated'])
#df

In [ ]:
# Normalize time (slowdown) and energy in comparison to the reference config for that CPU.

# Calculate normalized metrics for each run.
df_norm = df.copy()
df_norm['Ref SIMD FU'] = df_norm['CPU'].replace(ref_fu)
df_norm['Ref Mode'] = ref_mode
df_norm = df_norm.merge(df, left_on=['Benchmark', 'CPU', 'Width Block', 'Ref SIMD FU', 'Ref Mode'],
                        right_on=['Benchmark', 'CPU', 'Width Block', 'SIMD FU', 'Mode'],
                        suffixes=('', ' ref'))
df_norm['Time'] = df_norm['Time'] / df_norm['Time ref']
df_norm['Dynamic']   = df_norm['Dynamic'] / df_norm['Dynamic ref']
df_norm['Leakage']   = df_norm['Leakage'] / df_norm['Leakage ref']
df_norm['Total']     = df_norm['Total'] / df_norm['Total ref']
df_norm = df_norm.drop(columns=['Ref SIMD FU', 'Ref Mode', 'SIMD FU ref', 'Mode ref', 'Time ref',
                                'Leakage ref', 'Dynamic ref', 'Total ref'])
df_norm = pd.melt(df_norm, id_vars=['Benchmark', 'CPU', 'SIMD FU', 'Mode', 'Width Block'],
                  var_name='Metric', value_name='Value').copy()
df_norm = df_norm.set_index(['Benchmark', 'CPU', 'SIMD FU', 'Mode', 'Width Block', 'Metric'])
df_norm

# Apply geometric mean to reduce several runs of the same benchmark.
df_norm = df_norm.groupby(by=[benchmarks, None, None, None, None, None],
                          level=['Benchmark', 'CPU', 'SIMD FU', 'Mode', 'Width Block', 'Metric'])['Value'].apply(lambda group: gmean(group).mean()).reset_index().set_index(['Benchmark', 'CPU', 'SIMD FU', 'Mode', 'Width Block', 'Metric'])

# Create 'average' benchmark using the geometric mean.
average = df_norm.groupby(['CPU', 'SIMD FU', 'Mode', 'Width Block', 'Metric'], as_index=True, sort=False)['Value'].apply(lambda group: gmean(group).mean()).reset_index()
average['Benchmark'] = 'average'
average = average.set_index(['Benchmark', 'CPU', 'SIMD FU', 'Mode', 'Width Block', 'Metric'])
df_norm = pd.concat([df_norm, average])


# Calculate reduction % for energy metrics.  
df_norm.loc(axis=0)[pd.IndexSlice[:,:,:,:,:,['Dynamic', 'Leakage', 'Total']]] = \
    100 * (1 - df_norm.loc(axis=0)[pd.IndexSlice[:,:,:,:,:,['Dynamic', 'Leakage', 'Total']]])

df_norm

In [ ]:
# Save results as csv.

#df_norm.reset_index().set_index(['CPU', 'SIMD FU', 'Mode', 'Width Block', 'Benchmark', 'Metric']).sort_index().to_csv('resuts_diffenergy_time.csv')

In [ ]:
# Help function for plots.

def filter_data(df, bench=None, cpu=None, metric=None, wBlock=None, fu=None, mode=None):
    if bench is not None:
        df = df[df['Benchmark'].isin(bench)]
    if cpu is not None:
        df = df[df['CPU'].isin(cpu)]
    if metric is not None:
        df = df[df['Metric'].isin(metric)]
    if wBlock is not None:
        df = df[df['Width Block'].isin(wBlock)]
    if fu is not None:
        if type(fu) is list:
            df = df[df['SIMD FU'].isin(fu)]
        elif type(fu) is dict:
            df = df[df['SIMD FU'] == df['CPU'].replace(fu)]
    if mode is not None:
        df = df[df['Mode'].isin(mode)]
    
    return df


def get_chart(df, bench=None, cpu=None, metric=None, wBlock=None, fu=None, mode=None, domain=None, scheme='category10'):
    df_metric = filter_data(df, bench=bench, cpu=cpu, metric=metric, wBlock=wBlock, fu=fu, mode=mode)
        
    df_metric['Metric'] = df_metric['Metric'].replace(labels)
    
    ct = alt.Chart(df_metric).mark_line(point=True).encode(
        x=alt.X(
            'Benchmark:N',
            title="",
            axis=alt.Axis(labelAngle=-20),
            sort=bench_order
        ),
        y=alt.Y(
            'Value:Q',
            scale=alt.Scale(zero=False) if domain is None else alt.Scale(domain=domain),
            title=""
        ),
        color=alt.Color(
            'Configuration:N',
            scale=alt.Scale(range=scheme),
            sort=config_order
        ),
        row=alt.Row(
            'Metric:N',
            title=""
        )
    ).properties(
        height=200,
        width=800
    )
    
    return ct


def configure_chart(ct):
    ct = ct.configure_axis(
        labelFontSize=15,
        titleFontSize=16,
        grid=True
    ).configure_header(
        labelFontSize=15,
        titleFontSize=18
    ).configure_legend(
        orient='right',
        labelLimit=0,
        titleFontSize=18,
        labelFontSize=16,
        symbolType='stroke',
        symbolSize=800,
        symbolStrokeWidth=5
    ).configure_line(
        size=2.5
    ).configure_point(
        size=75
    )
    return ct

In [ ]:
# Add config column.
df_plot = df_norm.reset_index().copy()
df_plot['Configuration'] = df_plot['CPU'] + '-' + df_plot['Mode'] + '-' + df_plot['SIMD FU'].astype(str) + 'FU'
df_plot['Config Order'] = df_plot['Mode'].replace(modes_order) * df_plot['SIMD FU'].max() + (df_plot['SIMD FU'].max()-df_plot['SIMD FU'])
df_plot

In [ ]:
# Get configuration order.

config_order = list(df_plot.sort_values(['CPU', 'Config Order'])['Configuration'].unique())
config_order

In [ ]:
# Scales.

time_scale = (0.5, 3)
energy_scale = (-20, 70)

In [ ]:
# Plots A76, no penalty.

scheme = ['#ffaa73', '#a84400',
          '#95baff', '#005dff',
          '#84cd84', '#007700']
ct_nopen_A76 = get_chart(df_plot, cpu=['A76'], metric=['Dynamic'], wBlock=[8], mode=modes_nopenalty, domain=energy_scale, scheme=scheme) & \
               get_chart(df_plot, cpu=['A76'], metric=['Leakage'], wBlock=[8], mode=modes_nopenalty, domain=energy_scale, scheme=scheme) & \
               get_chart(df_plot, cpu=['A76'], metric=['Total'], wBlock=[8], mode=modes_nopenalty, domain=energy_scale, scheme=scheme) & \
               get_chart(df_plot, cpu=['A76'], metric=['Time'], wBlock=[8], mode=modes_nopenalty, domain=time_scale, scheme=scheme)
ct_nopen_A76 = configure_chart(ct_nopen_A76)
ct_nopen_A76

In [ ]:
# Plots A76, penalty.

scheme = ['#ffaa73', '#a84400',
          '#95baff', '#005dff',
          '#84cd84', '#007700']
ct_pen_A76 = get_chart(df_plot, cpu=['A76'], metric=['Dynamic'], wBlock=[8], mode=modes_penalty, domain=energy_scale, scheme=scheme) & \
             get_chart(df_plot, cpu=['A76'], metric=['Leakage'], wBlock=[8], mode=modes_penalty, domain=energy_scale, scheme=scheme) & \
             get_chart(df_plot, cpu=['A76'], metric=['Total'], wBlock=[8], mode=modes_penalty, domain=energy_scale, scheme=scheme) & \
             get_chart(df_plot, cpu=['A76'], metric=['Time'], wBlock=[8], mode=modes_penalty, domain=time_scale, scheme=scheme)
ct_pen_A76 = configure_chart(ct_pen_A76)
ct_pen_A76

In [ ]:
# Save A76 plot.

#ct_nopen_A76.save('plots/a76_diffenergy_time_nopen.svg', webdriver='firefox')
#ct_pen_A76.save('plots/a76_diffenergy_time_pen.svg', webdriver='firefox')

In [ ]:
# Plots HP, no penalty.

scheme = ['#ffc9a5', '#ff8533', '#8c3800',
          '#c5d2e9', '#5ea1ff', '#004cd2',
          '#9ae59a', '#3abf3a', '#006000']
ct_nopen_HP = get_chart(df_plot, cpu=['HP'], metric=['Dynamic'], wBlock=[8], mode=modes_nopenalty, domain=energy_scale, scheme=scheme) & \
              get_chart(df_plot, cpu=['HP'], metric=['Leakage'], wBlock=[8], mode=modes_nopenalty, domain=energy_scale, scheme=scheme) & \
              get_chart(df_plot, cpu=['HP'], metric=['Total'], wBlock=[8], mode=modes_nopenalty, domain=energy_scale, scheme=scheme) & \
              get_chart(df_plot, cpu=['HP'], metric=['Time'], wBlock=[8], mode=modes_nopenalty, domain=time_scale, scheme=scheme)
ct_nopen_HP = configure_chart(ct_nopen_HP)
ct_nopen_HP

In [ ]:
# Plots HP, penalty.

scheme = ['#ffc9a5', '#ff8533', '#8c3800',
          '#c5d2e9', '#5ea1ff', '#004cd2',
          '#9ae59a', '#3abf3a', '#006000']
ct_pen_HP = get_chart(df_plot, cpu=['HP'], metric=['Dynamic'], wBlock=[8], mode=modes_penalty, domain=energy_scale, scheme=scheme) & \
            get_chart(df_plot, cpu=['HP'], metric=['Leakage'], wBlock=[8], mode=modes_penalty, domain=energy_scale, scheme=scheme) & \
            get_chart(df_plot, cpu=['HP'], metric=['Total'], wBlock=[8], mode=modes_penalty, domain=energy_scale, scheme=scheme) & \
            get_chart(df_plot, cpu=['HP'], metric=['Time'], wBlock=[8], mode=modes_penalty, domain=time_scale, scheme=scheme)
ct_pen_HP = configure_chart(ct_pen_HP)
ct_pen_HP

In [ ]:
# Save HP plots.

#ct_nopen_HP.save('plots/hp_diffenergy_time_nopen.svg', webdriver='firefox')
#ct_pen_HP.save('plots/hp_diffenergy_time_pen.svg', webdriver='firefox')

In [ ]:
# Plots HP latency penalty impact.

scheme = ['#95baff', '#005dff',
          '#84cd84', '#007700']
ct_hp_penalty = get_chart(df_plot, cpu=['HP'], metric=['Time'], mode=['Packing', 'PackingPenalty', 'Fusing', 'FusingPenalty'], wBlock=[8], fu=ref_fu, domain=time_scale, scheme=scheme)
ct_hp_penalty = configure_chart(ct_hp_penalty)
ct_hp_penalty

In [ ]:
# Save latency penalty plot.

#ct_hp_penalty.save('plots/hp_time_compare.svg', webdriver='firefox')